In [228]:
import numpy as np
import pandas as pd


In [252]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = pd.read_csv('returns.csv', header=None)
# df = df.apply(lambda iterator: ((iterator - iterator.mean())/iterator.std()))

In [253]:
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.021732,0.023444,0.002508,-0.003098,0.009709,-0.000547,0.017476,0.015424,0.063718,0.015817,...,0.013942,0.002237,0.022508,0.018167,0.014880,-0.008362,0.005078,0.005232,0.017371,0.007844
1,0.005219,0.004288,0.003842,-0.009322,0.017449,0.018053,-0.003690,0.016591,0.003524,-0.001342,...,0.015743,-0.011905,0.003241,0.020038,-0.022159,-0.020699,-0.002210,-0.004673,0.005155,-0.024625
2,0.008180,0.013026,-0.003115,-0.008234,0.006754,0.001612,0.011543,-0.009042,0.018258,0.012455,...,0.020796,0.004217,0.013256,0.013724,0.011927,-0.005950,-0.002215,0.008644,0.009615,0.006550
3,-0.000808,-0.008324,0.029286,0.024115,0.003865,-0.004292,0.007273,-0.009681,-0.004138,-0.025046,...,0.012301,0.008698,0.000836,-0.025285,-0.006735,-0.007245,-0.008246,-0.010368,0.018730,-0.008992
4,0.007161,-0.001991,-0.007634,0.015634,0.008801,-0.007004,0.004792,0.007360,-0.013158,-0.003450,...,-0.017847,0.001784,-0.002702,-0.008280,0.002882,-0.001745,-0.002558,0.002031,0.002181,-0.002746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,-0.005340,-0.001371,-0.002687,0.002125,0.001843,-0.003877,-0.003474,-0.007024,-0.004037,-0.002903,...,0.010400,0.006996,0.002274,-0.005112,0.000573,-0.002977,-0.005046,-0.007297,-0.004043,-0.018258
1254,-0.011201,0.018724,0.005030,-0.007551,-0.006131,0.002271,0.021414,-0.001875,-0.008107,-0.000336,...,0.000606,0.005127,-0.001920,0.000856,-0.024889,-0.001148,-0.006174,-0.011531,-0.006203,0.009972
1255,0.017974,0.027799,0.010724,0.018162,0.012647,0.012298,0.014983,0.017163,0.008757,0.006497,...,0.026220,0.019188,0.006470,0.009840,0.018220,0.005059,0.018860,0.008895,0.011565,0.007052
1256,-0.013059,-0.007061,-0.004244,0.001812,-0.002437,-0.007353,-0.004610,-0.020232,-0.013310,-0.007457,...,-0.009237,-0.004295,-0.014767,-0.009320,-0.004865,-0.003203,-0.005246,-0.007226,-0.011614,-0.015717


In [274]:
# ai
def sample_cov(df):
    cov = np.array(df - df.mean(axis=0))
    total = np.zeros((cov.shape[1], cov.shape[1]))
    for r in range(cov.shape[0]):
        total += cov[[r], :].T @ cov[[r], :]
    return total/cov.shape[0]


sample_covariance = sample_cov(df)
np.savetxt('sample_covariance.csv', sample_covariance, delimiter=',')


In [270]:
def run(S, lam=0.00001, iterations = 10000, stopping_thres = 0):
    T, N = df.shape
    J = np.eye(N)
    def assign(i,j, new):
        J[i][i] = (new)[0][0]
        J[i][j] = (new)[0][1]
        J[j][i] = (new)[1][0]
        J[j][j] = (new)[1][1]
    def obj():
        return np.log(np.linalg.det(J))-np.trace(J @ S) - lam * np.sum(np.abs(J - np.diag(np.diag(J))))
        # return np.log(np.linalg.det(J))-np.trace(J @ S) - lam * (sum(sum(abs(J))) - np.trace(abs(J)))
    prev_obj = obj()
    for _ in range(iterations):
        for i in range(N):
            for j in range(i+1,N):
                Jbb = J[[a for a in range(N) if not a in [i,j]], :][:, [a for a in range(N) if not a in [i,j]]]
                Jab = J[[i, j], :][:, [a for a in range(N) if not a in [i,j]]]
                Jba = J[[a for a in range(N) if not a in [i,j]], :][:, [i, j]]
                L = Jab@np.linalg.inv(Jbb)@Jba
                Saa = S[[i, j], :][:, [i, j]]
                Svv = S[j][j]
                Suu = S[i][i]
                ms = []
                try: 
                    m1 = np.linalg.inv(Saa+lam*np.array([[0, 1], [1, 0]]))
                except: 
                    pass
                else:
                    if np.all(np.linalg.eigvals(m1) > 0):
                        ms.append(m1)
                try: 
                    m2 = np.linalg.inv(Saa+lam*np.array([[0, -1], [-1, 0]]))
                except: 
                    pass
                else:
                    if np.all(np.linalg.eigvals(m2) > 0):
                        ms.append(m2)
                try: 
                    m3 = np.array([[(1+np.sqrt(1+4*Svv*Suu*L[0][1]**2))/(2*Suu), -L[0][1]],
                                [-L[1][0], (1+np.sqrt(1+4*Svv*Suu*L[0][1]**2))/(2*Svv)]])
                except: 
                    pass
                else:
                    if np.all(np.linalg.eigvals(m3) > 0) and np.any(np.isnan(m3)):
                        ms.append(m3)
                curr_obj = obj()
                for m in ms:
                    prev_Jaa = J[[i,j],:][:,[i,j]]
                    assign(i, j, m+L)
                    new_obj = obj()
                    if new_obj>curr_obj:
                        curr_obj = new_obj
                    else:
                        assign(i, j, prev_Jaa)
        # if obj()-prev_obj < stopping_thres:
            # break
        if curr_obj-prev_obj > stopping_thres:
            prev_obj = curr_obj
            print(curr_obj)
        else:
            break
    print(np.count_nonzero(J))
    return J


In [271]:
J_test = run(sample_covariance, 0, 1)

385.0056759997497
2500


In [272]:
new_J = run(sample_covariance, 0.00001)

380.4197530017843
391.0326624128037
394.1786065074648
395.3742950060432
395.88432341540766
396.26606417195796
396.6505194282163
397.01563621991335
397.3463916676511
397.6223324398686
397.86358348742414
398.0631110785621
398.2164931587084
398.32723163974885
398.41790228705094
398.4693825025649
398.51013741665287
398.5349459835055
398.5573109614745
398.57676925066045
398.5851492784454
398.591094718401
398.59511073704283
398.5987347747252
398.60207692292
398.604096897465
398.60541623694223
398.606222102745
398.60674791121056
398.6071651829852
398.60760241591043
398.6079547510949
398.60817726214196
398.60839552713975
398.6085543237606
398.6086528369176
398.60873079749734
398.60879243628966
398.60884212886276
398.6088812176039
398.60891292230065
398.6089450873016
398.6089661444202
398.60898291851254
398.6089963185406
398.6090070463206
398.6090155408171
398.60902230804885
398.6090277133595
398.60903206409665
398.60903559628537
398.60903848816525
398.6090408547387
398.6090427977645
398.609044

In [273]:
new_J1 = run(sample_covariance, 0.0001)

362.3997860189677
363.68106019009815
363.9072771542286
364.0166969285136
364.1188795927008
364.14068152791117
364.1510739248299
364.1580863216129
364.1636093685146
364.1661778994537
364.16755387925906
364.16791826797123
364.1679632315418
364.1679798737139
364.1679910978421
364.16800018817395
364.1680027557484
364.16800312650344
364.1680032746521
364.16800334400364
364.1680033804277
364.168003399888
364.16800340978716
364.16800341446185
364.1680034165032
364.16800341733875
364.1680034176731
364.1680034178134
364.16800341787626
364.16800341790804
364.1680034179245
364.16800341793146
364.1680034179356
364.1680034179369
364.1680034179376
364.16800341793817
364.1680034179389
778


In [259]:
new_J2 = run(sample_covariance, 0.001)

50


In [261]:
print(np.count_nonzero(new_J))
print(np.count_nonzero(new_J1))
print(np.count_nonzero(new_J2))

1996
778
50


In [260]:
print(np.count_nonzero(np.linalg.inv(new_J)))
print(np.count_nonzero(np.linalg.inv(new_J1)))
print(np.count_nonzero(np.linalg.inv(new_J2)))

2500
2500
50


In [275]:
from sklearn.covariance import graphical_lasso
# sample_covariance = df.cov().to_numpy()
S, J = graphical_lasso(sample_covariance, 0.00001)
print(np.count_nonzero(J)) # 1132
print(np.count_nonzero(S)) # 2500
S1, J1 = graphical_lasso(sample_covariance, 0.0001)
print(np.count_nonzero(J1)) # 538
print(np.count_nonzero(S1)) # 1942
S2, J2 = graphical_lasso(sample_covariance, 0.001)
print(np.count_nonzero(J2)) # 50
print(np.count_nonzero(S2)) # 50

1132
2500
538
1942
50
50


/Users/shuge/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.522957876285094e-11, tolerance: 4.448338758712089e-11
  coefs, _, _, _ = cd_fast.enet_coordinate_descent_gram(
/Users/shuge/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0171922562200253e-09, tolerance: 3.1977992399623245e-11
  coefs, _, _, _ = cd_fast.enet_coordinate_descent_gram(
/Users/shuge/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.561371455076433e-09, tolerance: 5.610078280262789e-11
  coefs, _, _, _ = cd_fast.enet_coordinate_descent_gram(
/Users/shuge/opt/anaconda3/lib/p